In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [2]:
# Helper Functions
# --------------------------------------------

def date_time(cell):
    """Extracts the date and time strings from a table cell."""
    return [item.strip() for item in list(cell.strings)][0:2]

def booster_version(cell):
    """Extracts the booster version string from the cell."""
    return ''.join([text for i, text in enumerate(cell.strings) if i % 2 == 0][:-1])

def landing_status(cell):
    """Returns the landing status string from the cell."""
    return list(cell.stripped_strings)[0]

def get_mass(cell):
    """Extracts and cleans the payload mass (e.g., '5000 kg')."""
    mass = unicodedata.normalize("NFKD", cell.text).strip()
    if "kg" in mass:
        return mass[:mass.find("kg")+2]
    return "0"

def extract_column_from_header(th):
    """Cleans header cell to extract a readable column name."""
    if th.br:
        th.br.extract()
    if th.a:
        th.a.extract()
    if th.sup:
        th.sup.extract()
    name = ' '.join(th.stripped_strings)
    if not name.isdigit():
        return name.strip()
    return None

In [3]:
# Static URL of the Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)  # Send GET request
soup = BeautifulSoup(response.text, 'html.parser')  # Parse HTML content
print(soup.title.string)  # Print page tit

List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [4]:
# Get all HTML tables
html_tables = soup.find_all("table", class_="wikitable plainrowheaders collapsible")

# Preview the third table (index 2)
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-2"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-2"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9

In [5]:
# Extract column names
column_names = []
for th in first_launch_table.find_all("th"):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)


['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [6]:
# Initialize dictionary with empty lists
launch_dict = dict.fromkeys(column_names)
del launch_dict['Date and time ( )']  # Remove irrelevant column

# Initialize each list in the dictionary
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Add extra fields
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

In [7]:
# Extract rows from tables
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
            else:
                flag = False
        else:
            flag = False

        row = rows.find_all('td')
        if flag and len(row) >= 9:
            extracted_row += 1
            try:
                # Flight Number
                launch_dict['Flight No.'].append(flight_number)

                # Date and Time
                datatimelist = date_time(row[0])
                date = datatimelist[0].strip(',')
                time = datatimelist[1]
                launch_dict['Date'].append(date)
                launch_dict['Time'].append(time)

                # Version Booster
                bv = booster_version(row[1])
                if not bv and row[1].a:
                    bv = row[1].a.string
                launch_dict['Version Booster'].append(bv)

                # Launch Site
                launch_site = row[2].a.string if row[2].a else row[2].text.strip()
                launch_dict['Launch site'].append(launch_site)

                # Payload
                payload = row[3].a.string if row[3].a else row[3].text.strip()
                launch_dict['Payload'].append(payload)

                # Payload Mass
                payload_mass = get_mass(row[4])
                launch_dict['Payload mass'].append(payload_mass)

                # Orbit
                orbit = row[5].a.string if row[5].a else row[5].text.strip()
                launch_dict['Orbit'].append(orbit)

                # Customer
                customer = row[6].a.string if row[6].a else row[6].text.strip()
                launch_dict['Customer'].append(customer)

                # Launch Outcome
                launch_outcome = list(row[7].strings)[0]
                launch_dict['Launch outcome'].append(launch_outcome)

                # Booster Landing
                booster_landing = landing_status(row[8])
                launch_dict['Booster landing'].append(booster_landing)

            except Exception as e:
                print(f"Error extracting row {extracted_row}: {e}")
                continue

In [8]:
# Convert to DataFrame
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Display sample of DataFrame
print(df.head())

  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          2       CCAFS                                Dragon            0   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome   Version Booster Booster landing  \
0   LEO   SpaceX      Success\n  F9 v1.07B0003.18         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.18         Failure   
2   LEO     NASA        Success  F9 v1.07B0005.18      No attempt   
3   LEO     NASA      Success\n  F9 v1.07B0006.18      No attempt   
4   LEO     NASA      Success\n  F9 v1.07B0007.18      No attempt   

              Date   Time  
0      4 June 2010  18:45  
1  8 December 2010  15:43  
2      22 May 2012  07:44 

In [9]:
df.to_csv('spacex_web_scraped.csv', index=False)